## ch 2. 데이터 전처리하기

크롤링을 통해 수집한 뉴스 기사 데이터에서 필요없는 부분을 먼저 제거하겠습니다. 텍스트 데이터를 전처리할 때는 정규 표현식을 활용하면 좋습니다.

## 불필요한 텍스트 제거
- 기사 시작 부분에 언론사나 기자 이름을 언급하는 부분
- 기사 마지막 부분에 사진 출처를 언급하는 부분

In [24]:
!pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable


In [107]:
#df = pd.read_csv("./data/baseball.csv", delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

In [191]:
!pip install tqdm

### 전처리 하기 전 텍스트 확인

In [81]:
import pandas as pd
import csv

baseball_df = pd.read_csv("./data/baseball.tsv", encoding='utf-8', sep="\t")

In [82]:
baseball_df["content"]

0         [스포티비뉴스=대구, 박성윤 기자] 메이저리그가 수비 시프트를 금지한다. 수비 시프...
1         (엑스포츠뉴스 고척, 박윤서 기자) 지옥과 천당을 오갔다.키움 히어로즈 장재영은 1...
2         이정후(24)가 "우리가 1등"이라고 자신한 키움 히어로즈 외야에 기분 좋은 변수가...
3         [창원=스포츠조선 박상경 기자] 지난 12일 창원NC파크.KIA 타이거즈 황대인은 ...
4         [OSEN=대구, 이상학 기자] 류중일(59) 항저우 아시안게임 야구대표팀 감독은 ...
                                ...                        
102914    [스포츠경향] 프로야구 출범 40주년을 맞아 한국야구위원회(KBO)가 다시 한 번 ...
102915    [스포티비뉴스=고봉준 기자] KBO 정지택 총재가 새해를 맞아 야구팬들에게 인사를 ...
102916    [스포츠조선 박상경 기자] KBO 정지택 총재가 신년사를 발표했다.정 총재는 1일 ...
102917    LG 트윈스의 미래를 짊어질 좌완 영건 조원태(18)가 팀의 레전드 ‘야생마’ 이상...
102918    출범 40년을 넘긴 KBO가 2022년을 맞아 달라질 모습을 미리 전했다.정지택 K...
Name: content, Length: 102919, dtype: object

In [83]:
baseball_df.isnull().sum()

url           0
datetime      0
title         0
content     399
dtype: int64

In [84]:
baseball_df = baseball_df.dropna()

## 필터링에 사용할 정규 표현식 작성 요령

1. 예시 데이터를 가지고 정규표현식을 작성한 뒤, 결과를 확인한다.
2. 여러 데이터 샘플들에 대해서도 정규표현식이 의도한 대로 동작하는지 확인한다.

In [85]:
example_content = baseball_df["content"][0]

In [86]:
example_content

'[스포티비뉴스=대구, 박성윤 기자] 메이저리그가 수비 시프트를 금지한다. 수비 시프트를 사랑하는 한화 이글스 카를로스 수베로 감독은 아쉬워하면서도 잃는 게 있으면 얻는 것도 있다고 설명했다.한화 수베로 감독은 13일 대구삼성라이온즈파크에서 삼성 라이온즈와 시범경기에 앞서 메이저리그 수비 시프트 금지에 대한 본인 생각을 이야기헀다. 메이저리그는 오는 2023년부터 수비 시프트를 금지한다. 새로운 CBA(노사협정) 개정으로 규칙이 바뀐 셈이다.KBO 리그에 변화는 없다. 그러나 메이저리그를 경험해본 수베로 감독은 시프트가 사라지는 것이 아쉽다. 그는 KBO 리그에서 가장 많은 시프트를 사용하는 감독 가운데 한 명이다. 타자 유형에 따라 공격적인 시프트를 보여준다. 2루수가 유격수 쪽에 치우치거나 우익수 쪽으로 나가는 것은 기본적인 시프트일 정도로 다양한 시프트를 활용해 타자의 안타 생산을 막는다.13일 수베로 감독은 "나는 수비 시프트를 좋아한다. 시프트는 야구의 일부다. 야구는 확률 싸움을 하는 스포츠다. 시프트는 확률에 따라 맞춰 움직여 아웃 카운트를 잡을 수 있다. 마음에 든다. 최근 생긴 방법도 아니다. 1980~1990년대부터 경기를 할 때 투수와 타자에 맞춰 수비 위치를 바꿨다. 투수의 투구와 타자의 타격 성향을 고려해 움직였다. 나는 시프트 자체를 높게 평가한다"며 야구를 잘하는 수단으로 시프트가 좋다고 목소리를 높였다.메이저리그에는 수베로 감독만큼 시프트를 즐겨 사용하는 감독들이 많다. 극단적으로 당겨치는 왼손 타자를 사대하기 위해 내야수 4명 전원을 1루와 2루 사이에 배치한 장면도 있다. 메이저리그 강타자 배리 본즈는 시프트가 있어도 신경 쓰지 않고 본인의 타격을 한 것으로 유명하다. 투수들은 간혹 정상 수비였다면 아웃되는 타구였는데, 시프트로 안타를 내주자 아쉬워하거나 화를 내기도 한다. 시프트는 야구의 전술 혹은 문화로 자리를 잡았는데, 이를 금지해 반기거나 아쉬워하는 두 종류의 소리가 나오고 있다.하나의 전술로 자리 잡은 시프트의 금지는 

In [87]:
import re
# 정규표현식 (가장 첫 시작이 대괄호, .+모든 문자, 하나보다 더 많이 있는 글자, 가장 먼저 있는 대괄호만 패턴화하기)
pattern = r"^\[.+?\]"

In [88]:
re.findall(pattern, example_content)

['[스포티비뉴스=대구, 박성윤 기자]']

In [89]:
# 정규표현식 패턴을 찾아서 비어있는 문자열로 대체하기
preprocessed_content = re.sub(pattern, "", example_content)

In [90]:
preprocessed_content = preprocessed_content.strip()
preprocessed_content

'메이저리그가 수비 시프트를 금지한다. 수비 시프트를 사랑하는 한화 이글스 카를로스 수베로 감독은 아쉬워하면서도 잃는 게 있으면 얻는 것도 있다고 설명했다.한화 수베로 감독은 13일 대구삼성라이온즈파크에서 삼성 라이온즈와 시범경기에 앞서 메이저리그 수비 시프트 금지에 대한 본인 생각을 이야기헀다. 메이저리그는 오는 2023년부터 수비 시프트를 금지한다. 새로운 CBA(노사협정) 개정으로 규칙이 바뀐 셈이다.KBO 리그에 변화는 없다. 그러나 메이저리그를 경험해본 수베로 감독은 시프트가 사라지는 것이 아쉽다. 그는 KBO 리그에서 가장 많은 시프트를 사용하는 감독 가운데 한 명이다. 타자 유형에 따라 공격적인 시프트를 보여준다. 2루수가 유격수 쪽에 치우치거나 우익수 쪽으로 나가는 것은 기본적인 시프트일 정도로 다양한 시프트를 활용해 타자의 안타 생산을 막는다.13일 수베로 감독은 "나는 수비 시프트를 좋아한다. 시프트는 야구의 일부다. 야구는 확률 싸움을 하는 스포츠다. 시프트는 확률에 따라 맞춰 움직여 아웃 카운트를 잡을 수 있다. 마음에 든다. 최근 생긴 방법도 아니다. 1980~1990년대부터 경기를 할 때 투수와 타자에 맞춰 수비 위치를 바꿨다. 투수의 투구와 타자의 타격 성향을 고려해 움직였다. 나는 시프트 자체를 높게 평가한다"며 야구를 잘하는 수단으로 시프트가 좋다고 목소리를 높였다.메이저리그에는 수베로 감독만큼 시프트를 즐겨 사용하는 감독들이 많다. 극단적으로 당겨치는 왼손 타자를 사대하기 위해 내야수 4명 전원을 1루와 2루 사이에 배치한 장면도 있다. 메이저리그 강타자 배리 본즈는 시프트가 있어도 신경 쓰지 않고 본인의 타격을 한 것으로 유명하다. 투수들은 간혹 정상 수비였다면 아웃되는 타구였는데, 시프트로 안타를 내주자 아쉬워하거나 화를 내기도 한다. 시프트는 야구의 전술 혹은 문화로 자리를 잡았는데, 이를 금지해 반기거나 아쉬워하는 두 종류의 소리가 나오고 있다.하나의 전술로 자리 잡은 시프트의 금지는 야구의 퇴행일까. 시프트를 사랑하는 

### 전체 데이터 셋에서 일부를 샘플링해서 정규표현식 패턴에 일치하는 텍스트만 출력

In [91]:
sampled_df = baseball_df.sample(1000, random_state=1234)

In [92]:
for i in range(len(sampled_df)):
    row = sampled_df.iloc[i]
    content = row["content"]
    matches = re.findall(pattern, content)
    if matches:
        print(matches)

['[부산=스포츠조선 김영록 기자]']
['[마이데일리 = 윤욱재 기자]']
['[스포티비뉴스=김민경 기자]']
['[스포츠조선 권인하 기자]']
['[OSEN=대구, 이상학 기자]']
['[스포티비뉴스=신원철 기자]']
['[스포티비뉴스=광주, 김민경 기자]']
['[OSEN=잠실, 이후광 기자]']
['[OSEN=이천, 한용섭 기자]']
['[OSEN=대구, 손찬익 기자]']
['[스포티비뉴스=고척, 고봉준 기자]']
['[2022 KBO리그 기록실]']
['[주장]']
['[OSEN=길준영 기자]']
['[스포티비뉴스=인천, 최민우 기자]']
['[스포티비뉴스=잠실, 박정현 기자]']
['[KBO리그]']
['[OSEN=대구, 손찬익 기자]']
['[스포츠춘추]']
['[스포티비뉴스=잠실, 박정현 기자]']
['[2022 KBO리그 기록실]']
['[창원=스포츠조선 김 용 기자]']
['[마이데일리 = 김건호 기자]']
['[창원=스포츠조선 박상경 기자]']
['[데일리안 = 김수민 기자]']
['[OSEN=거제, 이상학 기자]']
['[일간스포츠 안희수]']
['[스포티비뉴스=수원, 고봉준 기자]']
['[스포티비뉴스=잠실, 박성윤 기자]']
['[마이데일리 = 광주 윤욱재 기자]']
['[스포츠경향]']
['[OSEN=고척, 길준영 기자]']
['[잠실=김은진 기자]']
['[OSEN=수원, 한용섭 기자]']
['[스포츠조선 이종서 기자]']
['[마이데일리 = 윤욱재 기자]']
['[OSEN=고척, 길준영 기자]']
['[스포츠조선 이종서 기자]']
['[서울=뉴시스]']
['[광주=스포츠조선 김영록 기자]']
['[스포티비뉴스=고봉준 기자]']
['[스포츠조선 박상경 기자]']
['[광주=스포츠조선 박상경 기자]']
['[서울=뉴시스]']
['[OSEN=고흥, 길준영 기자]']
['[KBO리그]']
['[스포츠조선 박상경 기자]']
['[마이데일리 = 윤욱재 기자]']
['[스포츠조선 이종서 기자]']
['[서울=뉴시스]']
['[고척=스포츠조선 

### re.sub 함수를 이용해서 불필요한 텍스트 제거
앞서서 검증한 패턴을 가지고 데이터 셋에서 불필요한 부분을 제거

In [93]:
def preprocess(content):
    preprocessed_content = re.sub(pattern, "", content)
    return preprocessed_content.strip()

sampled_df["content"] = sampled_df["content"].apply(lambda x: preprocess(x))
# apply - 하나씩 꺼내서 preprocess 적용시켜주고 다시 넣어주겠다

In [94]:
sampled_df

,url,datetime,title,content
83655,https://sports.news.naver.com/news?oid=076&aid...,20220420,"""스윙에 문제 있다"" 서른살 아들도 피하지 못한 '모두까기' 비판 [부산초점]","아버지와 달리 슈퍼스타로는 성장하지 못했다. 하지만 서른까지 리그에 살아남았고, 자..."
88302,https://sports.news.naver.com/news?oid=117&aid...,20220401,KIA 김도영만 신인왕 후보 아니다…개막 엔트리 합격자 무려 11명,KIA '슈퍼루키' 김도영(19)은 벌써부터 유력한 신인왕 후보로 거론되고 있지만 ...
35905,https://sports.news.naver.com/news?oid=001&aid...,20220907,투수는 제5의 내야수…고비마다 투수 실책에 우는 KIA,(서울=연합뉴스) 임순현 기자 = 흔히들 야구에서 투수는 타자를 상대로 공을 던지는...
86455,https://sports.news.naver.com/news?oid=001&aid...,20220323,[모멘트] SSG랜더스 라커룸입니다,(인천=연합뉴스) 한상균 기자 = 프로야구 SSG랜더스가 23일 언론에 공개한 홈구...
17267,https://sports.news.naver.com/news?oid=001&aid...,20221020,"kt 소형준, 6이닝 2실점 5K 호투…준PO서도 '대형준' 이름값",1·3회 실점에도 평정심 유지…5·6회 연속 삼자범퇴 (수원=연합뉴스) 홍규빈...
...,...,...,...,...
77890,https://sports.news.naver.com/news?oid=144&aid...,20220427,"'첫 실전 등판' 김원중 ""빨리 1군에 오를 수 있도록 노력하겠다""[스경X코멘트]",롯데 ‘장발’의 마무리 투수 김원중(29)이 복귀 과정을 차근차근 밟아가고 있다.김...
37916,https://sports.news.naver.com/news?oid=109&aid...,20220911,13G 중 키움 상대 4G…벤자민의 'ERA 0.78' 극강 비결은? [오!쎈 고척],KT 위즈 외국인 투수 웨스 벤자민이 승수 사냥에 성공했다. 키움 히어로즈전 강세도...
82690,https://sports.news.naver.com/news?oid=109&aid...,20220416,"""10연승 하면 시구"" 용진이 형, SSG팬과 한 약속 지켰다",정용진 SSG 랜더스 구단주가 “10연승 달성 시 홈경기 시구를 하겠다”는 팬들과 ...
75835,https://sports.news.naver.com/news?oid=076&aid...,20220513,'영빈아 비켜봐~' 국대 중견수 박해민 '내야 실력도 훈수 둘 수준' [잠실 현장] ',국가대표 중견수의 내야수비 실력은?침체의 그림자를 걷어낸 박해민이 이번에는 내야 수...


In [ ]:
# 샘플로 확인해봤으니 전체 데이터에 적용시키기

In [95]:
# 진행상황 시각화 - 판다스, 자연어 처리할 때때
from tqdm import tqdm

tqdm.pandas()
baseball_df["content"] = baseball_df["content"].progress_apply(lambda x: preprocess(x))

100%|██████████████████████████████████████████████████████████████████████| 102520/102520 [00:00<00:00, 166264.23it/s]


In [96]:
#데이터 전처리 후 결측치 확인
baseball_df.isnull().sum()

url         0
datetime    0
title       0
content     0
dtype: int64

### 날짜 순으로 정렬

In [99]:
baseball_df.sort_values(["datetime"])

,url,datetime,title,content
102918,https://sports.news.naver.com/news?oid=108&aid...,20220101,"KBO 정지택 총재 ""S존 개선하겠다... 메타버스-NFT 개발도""",출범 40년을 넘긴 KBO가 2022년을 맞아 달라질 모습을 미리 전했다.정지택 K...
102885,https://sports.news.naver.com/news?oid=117&aid...,20220101,"""이용규 선배님께 죄송"" 데뷔도 안한 LG 신인이 고개숙인 사연","""죄송하다는 말씀을 꼭 드리고 싶어요""아직 프로 무대에 데뷔도 하지 않은 신인 선수..."
102884,https://sports.news.naver.com/news?oid=119&aid...,20220101,"KBO 정지택 총재 ""S존 유연하게 적용...NFT 개발 준비""",KBO총재 신년사 통해 2022년 계획과 추진방향 전달[데일리안 = 김태훈 기자] ...
102883,https://sports.news.naver.com/news?oid=109&aid...,20220101,"박병호-손아섭 이적에 뿔난 팬심, 냉철한 구단…감성과 이성의 충돌",올 겨울 KBO리그 FA 시장의 주목할 만한 특징은 프랜차이즈 스타들의 무더기 이적...
102882,https://sports.news.naver.com/news?oid=109&aid...,20220101,"""힘든 시기 팬들 덕분에 행복&다시 오지 않는 추억"" 스타들의 새해 인사",2022년 임인년(壬寅年) 새해가 밝았다. 야구팬들의 응원을 받는 KBO 리그 선수...
...,...,...,...,...
2067,https://sports.news.naver.com/news?oid=657&aid...,20221231,"삼성라이온즈, 오키나와 전지훈련···유일하게 '일본'에서만 캠프 진행",코로나 19 이후 처음으로 프로야구 10개 구단 무려 6개 구단이 모이는 애리조나를...
2068,https://sports.news.naver.com/news?oid=076&aid...,20221231,"505이닝→994.2이닝 클래스 입증, 안방 마님 두번째 FA는 대박치나[SC초점]",키움 히어로즈 이지영(36)에겐 주전 포수로서 가치를 증명한 한 해였다.지난 시즌 ...
2069,https://sports.news.naver.com/news?oid=117&aid...,20221231,"단장의 바람은 절반만 현실로…LG 31승 듀오, 2023년 우승청부사 기대","“두 사람이 30승을 만들어주는 게 바람이다."" LG 차명석 단장은 지난 1월 말에..."
2071,https://sports.news.naver.com/news?oid=047&aid...,20221231,"프로야구 40주년 결산, 환희와 아쉬움의 순간들",2019년 이후 3년 만에 600만 관중 달성2022년은 한국프로야구(KBO)가 출...


In [100]:
baseball_df = baseball_df.sort_values(["datetime"])

In [101]:
baseball_df.to_csv("./data/baseball_preprocessed.csv", index=False, encoding="utf-8", sep="\t")

### 결과 출력

In [102]:
preprocessed_df = pd.read_csv("./data/baseball_preprocessed.csv", encoding="utf-8", sep="\t")

In [103]:
preprocessed_df

,url,datetime,title,content
0,https://sports.news.naver.com/news?oid=108&aid...,20220101,"KBO 정지택 총재 ""S존 개선하겠다... 메타버스-NFT 개발도""",출범 40년을 넘긴 KBO가 2022년을 맞아 달라질 모습을 미리 전했다.정지택 K...
1,https://sports.news.naver.com/news?oid=117&aid...,20220101,"""이용규 선배님께 죄송"" 데뷔도 안한 LG 신인이 고개숙인 사연","""죄송하다는 말씀을 꼭 드리고 싶어요""아직 프로 무대에 데뷔도 하지 않은 신인 선수..."
2,https://sports.news.naver.com/news?oid=119&aid...,20220101,"KBO 정지택 총재 ""S존 유연하게 적용...NFT 개발 준비""",KBO총재 신년사 통해 2022년 계획과 추진방향 전달[데일리안 = 김태훈 기자] ...
3,https://sports.news.naver.com/news?oid=109&aid...,20220101,"박병호-손아섭 이적에 뿔난 팬심, 냉철한 구단…감성과 이성의 충돌",올 겨울 KBO리그 FA 시장의 주목할 만한 특징은 프랜차이즈 스타들의 무더기 이적...
4,https://sports.news.naver.com/news?oid=109&aid...,20220101,"""힘든 시기 팬들 덕분에 행복&다시 오지 않는 추억"" 스타들의 새해 인사",2022년 임인년(壬寅年) 새해가 밝았다. 야구팬들의 응원을 받는 KBO 리그 선수...
...,...,...,...,...
102515,https://sports.news.naver.com/news?oid=657&aid...,20221231,"삼성라이온즈, 오키나와 전지훈련···유일하게 '일본'에서만 캠프 진행",코로나 19 이후 처음으로 프로야구 10개 구단 무려 6개 구단이 모이는 애리조나를...
102516,https://sports.news.naver.com/news?oid=076&aid...,20221231,"505이닝→994.2이닝 클래스 입증, 안방 마님 두번째 FA는 대박치나[SC초점]",키움 히어로즈 이지영(36)에겐 주전 포수로서 가치를 증명한 한 해였다.지난 시즌 ...
102517,https://sports.news.naver.com/news?oid=117&aid...,20221231,"단장의 바람은 절반만 현실로…LG 31승 듀오, 2023년 우승청부사 기대","“두 사람이 30승을 만들어주는 게 바람이다."" LG 차명석 단장은 지난 1월 말에..."
102518,https://sports.news.naver.com/news?oid=047&aid...,20221231,"프로야구 40주년 결산, 환희와 아쉬움의 순간들",2019년 이후 3년 만에 600만 관중 달성2022년은 한국프로야구(KBO)가 출...
